In [1]:
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller
import subprocess
import shutil
from selenium import webdriver
from selenium.webdriver.common.by import By
from tqdm import tqdm
import time
import pandas as pd

In [2]:
from user_agent import generate_user_agent

user_agent = generate_user_agent(navigator='chrome')
print(user_agent)

Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.111 Safari/537.36


In [176]:
# 0. 환경설정 및 세팅
try:
    shutil.rmtree(r"c:\chrometemp")  #쿠키 / 캐쉬파일 삭제
except FileNotFoundError:
    pass

subprocess.Popen(r'C:\Program Files\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp"') # 디버거 크롬 구동
chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
option.add_argument('user-agent='+user_agent)

try:
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
except:
    chromedriver_autoinstaller.install(True)
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
driver.implicitly_wait(10)

C:\Users\NH\AppData\Local\Temp\ipykernel_17436\1235294787.py:14: DeprecationWarning:

executable_path has been deprecated, please pass in a Service object



In [177]:
from selenium import webdriver

In [178]:
def detailPagePaser():
    product_title = driver.find_element(By.CLASS_NAME, "product-top-info-title")
    product_reviews = driver.find_elements(By.CLASS_NAME, "product-review-content")
    star = driver.find_element(By.CLASS_NAME, "product-detail-satisfaction-score")

    product_title = product_title.text
    star = star.text
    product_reviews = [x.text for x in product_reviews]
    
    return product_title, star, product_reviews

In [179]:
detail_urls = []

In [180]:
page_num = 1
main_url = ''.format(page_num)
driver.get(main_url)
driver.get(main_url)

elements = driver.find_elements(By.CLASS_NAME,"product-item")

for idx, _ in enumerate(elements):
    driver.get(main_url) # 이거 할때마다 elements가 바뀜 element 순서로 코드 짜야함.
    driver.get(main_url)
    elements = driver.find_elements(By.CLASS_NAME,"product-item")
    time.sleep(2)
    elements[idx].click() # 브라우저 크기에 따라 클릭이 안되는 이슈가 있음.
    time.sleep(2)
    detail_url = driver.current_url
    detail_urls.append(detail_url)
    time.sleep(2)
    

WebDriverException: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=113.0.5672.126)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00478893+48451]
	(No symbol) [0x0040B8A1]
	(No symbol) [0x00315058]
	(No symbol) [0x00311843]
	(No symbol) [0x00308AC5]
	(No symbol) [0x00309B2A]
	(No symbol) [0x00308E23]
	(No symbol) [0x00308175]
	(No symbol) [0x0030810C]
	(No symbol) [0x00306D46]
	(No symbol) [0x003074A8]
	(No symbol) [0x003167F2]
	(No symbol) [0x0036CA61]
	(No symbol) [0x0035A2BC]
	(No symbol) [0x0036C482]
	(No symbol) [0x0035A0B6]
	(No symbol) [0x00337E08]
	(No symbol) [0x00338F2D]
	GetHandleVerifier [0x006D8E3A+2540266]
	GetHandleVerifier [0x00718959+2801161]
	GetHandleVerifier [0x0071295C+2776588]
	GetHandleVerifier [0x00502280+612144]
	(No symbol) [0x00414F6C]
	(No symbol) [0x004111D8]
	(No symbol) [0x004112BB]
	(No symbol) [0x00404857]
	BaseThreadInitThunk [0x76247D59+25]
	RtlInitializeExceptionChain [0x7720B74B+107]
	RtlClearBits [0x7720B6CF+191]


In [106]:
titles = []
reviews = []
stars = []
for url in tqdm(detail_urls):
    driver.get(url)
    driver.get(url)
    time.sleep(3)
    title, star ,review = detailPagePaser()
    titles.append(title)
    reviews.append(review)
    stars.append(star)

100%|██████████████████████████████████████████████████████████████████████████████████| 60/60 [09:27<00:00,  9.45s/it]


In [182]:
df = pd.DataFrame({'title':titles,'review':reviews,'star':stars})

In [202]:
import os
import pickle

In [203]:
path = os.getcwd()

In [206]:
with open(path + '/data/raw.pickle','wb') as f:
    pickle.dump(df,f)

In [207]:
df.to_csv(path + '/data/raw.csv')